In [5]:
#| default_exp state

In [6]:
#| export
import jax
import jax.numpy as jnp
from fastcore.test import test_eq, test_ne

from flax.struct import dataclass as flax_dataclass
from chewc.config import SimConfig
from chewc.structs import Population, SimParam

import warnings


In [7]:
#| export
#chewc/state.py

# ==============================================================================
# 2. DYNAMIC STATE: Carried and updated by the main simulation loop (lax.scan).
# ==============================================================================
@flax_dataclass(frozen=True)
class SimState:
    """
    The dynamic state of the simulation.

    This object is threaded through the `generation_step` function and contains
    all arrays and values that change from one generation to the next.
    Stored in the CARRY for lax.scan; It is designed to be lean and JIT-friendly.
    """
    # Core data arrays (pre-allocated to max_pop_size)
    geno: jnp.ndarray          # Shape: (max_pop_size, n_chr, ploidy, n_loci)
    ibd: jnp.ndarray           # Shape: (max_pop_size, n_chr, ploidy, n_loci)
    pheno: jnp.ndarray         # Shape: (max_pop_size, n_traits)
    bv: jnp.ndarray            # Shape: (max_pop_size, n_traits)
    is_active: jnp.ndarray     # Shape: (max_pop_size,)
    
    # Pedigree and Identifiers
    id: jnp.ndarray            # Shape: (max_pop_size,)
    mother: jnp.ndarray        # Shape: (max_pop_size,)
    father: jnp.ndarray        # Shape: (max_pop_size,)
    gen: jnp.ndarray           # Shape: (max_pop_size,)

    # Core state pointers
    key: jax.random.PRNGKey
    write_pos: int
    gen_idx: int
    next_id: int


def _round_up_to_multiple(x: int, m: int) -> int:
    return ((x + m - 1) // m) * m

def init_state_from_founders(
    key: jax.random.PRNGKey,
    founder_pop: Population,
    sp: SimParam,
    config: SimConfig,
) -> SimState:
    """
    Create the scan carry from a founder Population.
    - Keeps founders active.
    - Sets write_pos to the *next cohort boundary*.
    """

    # block to validate or find correct value for padding
    original_max_pop_size = config.max_pop_size
    required_max_pop_size = _round_up_to_multiple(config.max_pop_size, config.population_size)
    if original_max_pop_size != required_max_pop_size:
        warnings.warn(
            f"max_pop_size ({original_max_pop_size}) is not a multiple of "
            f"population_size ({config.population_size}). "
            f"Automatically increasing max_pop_size to {required_max_pop_size} for a clean ring buffer.",
            UserWarning
        )
        config = config.replace(max_pop_size=required_max_pop_size)

    n_founders = int(jnp.sum(founder_pop.is_active))

    # Make sure max_pop_size is a multiple of population_size (simplifies the ring)
    assert (config.max_pop_size % config.population_size) == 0, \
        "Set max_pop_size to a multiple of population_size for a clean ring."

    # Where the NEXT cohort will be written (advance to the next block boundary)
    next_block_start = _round_up_to_multiple(n_founders, config.population_size) % config.max_pop_size

    next_id = int(jnp.max(founder_pop.id)) + 1 if founder_pop.id.size > 0 else 0

    return SimState(
        geno=founder_pop.geno,
        ibd=founder_pop.ibd,
        pheno=founder_pop.pheno,
        bv=founder_pop.bv if founder_pop.bv is not None else jnp.full_like(founder_pop.pheno, jnp.nan),
        is_active=founder_pop.is_active,
        id=founder_pop.id,
        mother=founder_pop.mother,
        father=founder_pop.father,
        gen=founder_pop.gen,
        key=key,
        write_pos=next_block_start,
        gen_idx=0,
        next_id=next_id,
    )


In [8]:
#| hide
import nbdev; nbdev.nbdev_export()